# Import related pacakages

In [13]:
import cobra
import sys
sys.path.append(r'./code/')
from cobrapy_ec_model_function import *

# Input and output files

In [12]:
#input files
# The genome-scale metabolic model for constructing the enzyme-constrained model
model_name = './model/iML1515_new.xml' 

# Reaction-kcat file.
# eg. AADDGT,64.41327046 #s-1
reaction_kcat_file = './data/nc_machine_learning_kapp.json'
select_key = 'kappmax_ensemble_model_s-1'

# Gene-abundance file organized from PAXdb
# eg. b0789,1.1
gene_abundance_file = "./data/gene_abundance.csv"

# Gene-molecular_weight file organized from EcoCyc
# eg. b3500,48.77251
gene_molecular_weight_file = "./data/gene_molecular_weight.csv"

reaction_gene_subunit_file = "./data/reaction_gene_subunit.csv"
c13reaction_file = './data/C13reaction.csv' 


#output files
gene_outfile = "./analysis/genes.csv"
gpr_outfile = "./analysis/all_reaction_GPR.csv"
reaction_gene_subunit_MW_file = "./analysis/reaction_gene_subunit_MW.csv"
reaction_MW_file = "./analysis/reaction_MW.csv"
reaction_kcat_MW_file = "./analysis/reaction_kcat_MW.csv"
json_output_file="./model/iML1515_irr_enz_constraint.json"
ECMpy_fluxes_outfile = './analysis/ECMpy_ori_solution_df_pfba.csv'
GEM_fluxes_outfile = './analysis/Orimodel_solution_df_pfba.csv'

# Step1: Preprocessing of model

The reversible reactions in the GEM model are divided into two irreversible reactions and splited isoenzyme. The input is iML1515 with 2713 reactions. The output is a model with 3356 irreversible reactions.

In [3]:
model = cobra.io.read_sbml_model(model_name)
convert_to_irreversible(model)
#split isoenzyme
model = isoenzyme_split(model)
model

Name,iML1515
Memory address,0x07f904f063130
Number of metabolites,1877
Number of reactions,5883
Number of groups,38
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


# Step2: Retrieving enzyme kinetics and proteomics data

The inputs are GEM model. The outputs are 'genes' and 'gpr_relationship' data in the iML1515.

In [4]:
[genes,gpr_relationship] = get_genes_and_gpr(model,gene_outfile,gpr_outfile)

Get the molecular weight of the enzyme (MW) according to the file of all_reaction_GPR.csv, which obtained from the previous step (gpr_relationship, ./analysis/all_reaction_GPR.csv). We need to manually correct the error of the gene_reaction_rule of a small amount of reactions in iML1515 (See Supplementary Table S1 for details), and also need to manually get the subunit of each protein from EcoCyc.

In [5]:
reaction_gene_subunit_MW = get_reaction_gene_subunit_MW(reaction_gene_subunit_file,gene_molecular_weight_file,reaction_gene_subunit_MW_file)
reaction_gene_subunit_MW.head(5)

,name,gene_reaction_rule,subunit_num,subunit_mw
ALATA_D2_num1,D-alanine transaminase,b2551,2,45.31659
ALATA_D2_num2,D-alanine transaminase,b0870,4,36.49471
SHCHD2,Sirohydrochlorin dehydrogenase (NAD),b3368,2,49.95140
CPPPGO,Coproporphyrinogen oxidase (O2 required),b2436,2,34.32263
GTHOr,Glutathione oxidoreductase,b3500,2,48.77251


In [6]:
# reaction-gene-subunit-MW file. 
# eg. ALATA_D2,D-alanine transaminase,b2551 or b0870,45.31659 or 36.49471 ,2 or 4 
reaction_MW = calculate_reaction_mw(reaction_gene_subunit_MW_file,reaction_MW_file)
reaction_MW.head(5)

,MW
ALATA_D2_num1,90.6332
ALATA_D2_num2,145.9788
SHCHD2,99.9028
CPPPGO,68.6453
GTHOr,97.545


Calculate kcat/MW. The inputs are 'reaction_kcat' and 'reaction_MW' data for calculating the kcat/MW (When the reaction is catalyzed by several isozymes, the maximum is retained).

In [7]:
reaction_kcat_mw = calculate_reaction_kcat_mw(reaction_kcat_file, reaction_MW_file, reaction_kcat_MW_file,select_key)
reaction_kcat_mw.head(5)

,MW,kcat,kcat_MW
ALATA_D2_num1,90.6332,9300.506998,102.616999
ALATA_D2_num2,145.9788,9300.506998,63.711354
SHCHD2,99.9028,1224.000000,12.251909
CPPPGO,68.6453,900.000000,13.110876
GTHOr,97.5450,25901.616143,265.535047


Calculate f. The input is 'genes' data, 'gene_abundance.csv' and 'gene_molecular_weight.csv'.

In [8]:
f = calculate_f(genes, gene_abundance_file, gene_molecular_weight_file)
f

0.4059986079578236

# Step3: Save enzyme concentration constraint model as json file.

enzyme model needed data

In [9]:
#The enzyme mass fraction 
f = 0.406
# The total protein fraction in cell.
ptot = 0.56 
# The approximated average saturation of enzyme.
sigma = 1 #kapp data sigma is 1
# Lowerbound  of enzyme concentration constraint. 
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

In [10]:
#create enzyme concentration constraint model
trans_model2enz_json_model_split_isoenzyme(model_name, reaction_kcat_MW_file, f, ptot, sigma, lowerbound, upperbound, json_output_file)

enz_model=get_enzyme_constraint_model(json_output_file)
enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,ECMpy_fluxes_outfile,reaction_kcat_MW_file)
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

norm_model = cobra.io.json.load_json_model(json_output_file)
norm_model_pfba_solution = cobra.flux_analysis.pfba(norm_model)
norm_model_pfba_solution_df = norm_model_pfba_solution.to_frame()
norm_model_pfba_solution_df.to_csv(GEM_fluxes_outfile)
print(norm_model_pfba_solution_df.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

#Compare with C13 data
c13reaction_2_enz_model_diff = get_diff_reaction_use_c13(c13reaction_file,enz_model_pfba_solution)
print (c13reaction_2_enz_model_diff)

0.3685101074308255
0.8697726420320147
['ME2', 'ICL', 'AKGDH', 'SUCOAS_reverse', 'SUCDi', 'ACKr_reverse', 'PTAr', 'MDH', 'PDH']
